In [36]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [37]:
df = pd.read_csv('dataset.csv')

In [38]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [39]:
#Retirando coluna id do usuario
df.drop(['customerID'], axis=1, inplace=True)

In [40]:
#removendo dados duplicados
dublicados = df.duplicated().sum()
print(dublicados)
df = df.drop_duplicates()


22


In [41]:
#aplicando label encoding

label_encoder = LabelEncoder()

colunas_binarias = ['Churn', 'gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

for coluna in colunas_binarias:
    df[coluna] = label_encoder.fit_transform(df[coluna])


In [42]:
#One hot encoding
df_train = pd.get_dummies(df)
df_train.head()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,Churn,MultipleLines_No,...,TotalCharges_995.35,TotalCharges_996.45,TotalCharges_996.85,TotalCharges_996.95,TotalCharges_997.65,TotalCharges_997.75,TotalCharges_998.1,TotalCharges_999.45,TotalCharges_999.8,TotalCharges_999.9
0,0,0,1,0,1,0,1,29.85,0,False,...,False,False,False,False,False,False,False,False,False,False
1,1,0,0,0,34,1,0,56.95,0,True,...,False,False,False,False,False,False,False,False,False,False
2,1,0,0,0,2,1,1,53.85,1,True,...,False,False,False,False,False,False,False,False,False,False
3,1,0,0,0,45,0,0,42.30,0,False,...,False,False,False,False,False,False,False,False,False,False
4,0,0,0,0,2,1,1,70.70,1,True,...,False,False,False,False,False,False,False,False,False,False


In [43]:
#verificando se possui coluna vazia
df_train.isnull().sum().sort_values(ascending=False).head(20)

TotalCharges_999.9     0
gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
tenure                 0
PhoneService           0
PaperlessBilling       0
MonthlyCharges         0
Churn                  0
MultipleLines_No       0
TotalCharges_973.35    0
TotalCharges_973.45    0
TotalCharges_973.55    0
TotalCharges_973.65    0
TotalCharges_973.95    0
TotalCharges_978       0
TotalCharges_978.6     0
TotalCharges_979.05    0
TotalCharges_97.5      0
dtype: int64

In [44]:
#saber quantas colunas tem meu dataframe
print(df_train.shape)

(7021, 6571)


In [46]:
X_train = df_train.iloc[:, 0:6570].values.astype(np.float32)
Y_train = df_train.iloc[:, 6570].values.astype(np.float32)


In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.25)

In [50]:
classificador = Sequential()
classificador.add(Dense(units =13141, activation = 'relu', input_dim = 6570))
classificador.add(Dense(units = 6570, activation = 'relu'))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dense(units = 1, activation = 'sigmoid'))
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['binary_accuracy'])
classificador.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 13141)          │    86,349,511 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6570)           │    86,342,940 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       841,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 173,533,668 (661.98 MB)

 Trainable params: 173,533,668 (661.98 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:
history = classificador.fit(X_train, Y_train, batch_size = 300,epochs = 100, verbose = 1, validation_split=0.2)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - binary_accuracy: 1.0000 - loss: 0.0000e+00 - val_binary_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - binary_accuracy: 1.0000 - loss: 0.0000e+00 - val_binary_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - binary_accuracy: 1.0000 - loss: 0.0000e+00 - val_binary_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/100
11/15 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - binary_accuracy: 1.0000 - loss: 0.0000e+00

KeyboardInterrupt: 